<a href="https://colab.research.google.com/github/LerinaMM/score_credito_clientes_data_science/blob/main/score_credito_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Passo a passo

1. Entender o desafio e a empresa
2. Importar a base de dados
3. Preparar a base de dados para o modelo
4. Criar um modelo de inteligencia artificial
  
  4.1 Score de credito: Ruim, médio e bom

5. Usar o modelo em um cenário real

##1. Conectar o Dataset a partir do drive

In [4]:
!wget -P "/content/drive/MyDrive/Datasets/Python IA/clientes.csv"

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


# 2. Importar Bibliotecas

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#3. Verificação do código

In [8]:
#Nomear o primeiro conjunto de dados

dados = pd.read_csv('/content/drive/MyDrive/Datasets/Python IA/clientes.csv')

In [10]:
dados.head()

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0


In [11]:
#Verificar as colunas da tabela e o tipo de cada coluna para começar a preparação da tabela para o modelo.

dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

#4. Preparar a base de dados para o modelo

In [18]:
#Para iniciar o modelo precisamos preparar a base de dados
#LabelEncoder ele pega a coluna e transforma os dados escritos em números, o LabelEnconder é um pacote do sklearn

#Uma opção seria importar dessa maneira
#import sklearn.preprocessing.LabelEnconder
#isso quer dizer que vamos importar uma parte do pacote

In [15]:
pip install scikit-learn

In [17]:
#isso quer dizer que vamos importar uma parte do pacote scikit-learn

from sklearn.preprocessing import LabelEncoder

In [19]:
#Vamos criar o codificador para cada coluna com a função fit_transform

codificador = LabelEncoder()
dados['profissao'] = codificador.fit_transform(dados['profissao'])
dados['mix_credito'] = codificador.fit_transform(dados['mix_credito'])
dados['comportamento_pagamento'] = codificador.fit_transform(dados['comportamento_pagamento'])

In [20]:
#Vamos verificar se as colunas foram transformadas para numeros

dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  int64  
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  int64  
 13  divida_total              100000 non-null  fl

#5. Separar as informações em dados de treino e dados de teste

In [21]:
#A tabela que vamos querer prever será o score de crédito (y)

y = dados['score_credito']

# Os dados que vamos querer utilizar serão todas as colunas que vamos realizar a previsão do score de credito
# Vamos utilizar a função .drop para retirar as colunas que não farão parte do modelo

# axis = 1 --> coluna
# axis = 0 --> linha

x = dados.drop(['score_credito','id_cliente'], axis=1) #esta excluindo duas colunas da tabela que não necessitaremos utilizar no modelo

from sklearn.model_selection import train_test_split

#Separa a base de dados nos 4 pedaços conforme separamos acima
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y)

#Se quiseres definir o tamanho dos dados de treino e teste, podes fazer dessa maneira

#x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, teste_size=0.3, random_state=1)

# Em torno de 70% dos dados utilizam para treino e 30% utilizam para teste

#6.Criar o modelo de inteligência artificial - Score de Crédito
Importa, cria e treina

In [22]:
#Importação dos modelos
#Modelo de Árvore de Decisão - e o Modelo KNN - Nearest neighbors

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [23]:
#Criação do modelo

modelo_arvoredecisao = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()

In [24]:
#Treinamento do modelo

modelo_arvoredecisao.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)

KNeighborsClassifier()